# Zero Shot Classification

In [1]:
# %pip install -q transformers torch scikit-learn

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Dataset

- dataset: [LittleFish-Coder/Fake_News_KDD2020](https://huggingface.co/datasets/LittleFish-Coder/Fake_News_KDD2020)

- real: 0
- fake: 1

In [4]:
from datasets import load_dataset, DatasetDict

# load and download the dataset from huggingface
dataset: DatasetDict = load_dataset("LittleFish-Coder/Fake_News_KDD2020", download_mode="reuse_cache_if_exists", cache_dir="dataset")  # type: ignore

Generating test split: 100%|██████████| 499/499 [00:00<00:00, 31633.36 examples/s]


In [5]:
# load the test dataset
test_dataset = dataset["test"]

In [6]:
test_dataset

Dataset({
    features: ['text', 'embeddings', 'label'],
    num_rows: 499
})

# Load Model

In [8]:
# Load Model
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# Test Model

In [9]:
id2label = {0: 'real', 1: 'fake'}
label2id = {'real': 0, 'fake': 1}

In [10]:
candidate_labels = ["real", "fake"]

In [12]:
results: list = classifier(test_dataset['text'], candidate_labels)  # type: ignore

In [13]:
pred_labels = []

for result in results:
    pred_labels.append(label2id[result['labels'][0]])

# Evaluate Model

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Ground Truth Labels
real_labels = torch.tensor(test_dataset['label'])
pred_labels = pred_labels

accuracy = accuracy_score(real_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.60
Precision: 0.59
Recall: 0.60
F1-score: 0.50
